# PyTorch ResNet18 Flexible Input Notebook

This notebook mirrors `resnet_model_arch_binary_classification_hikaru_Sep8.ipynb`, but re-implements the workflow in PyTorch using a ResNet18 backbone pretrained on ImageNet. It supports flexible combinations of imaging modalities (e.g., `B_mode`, `MBF`, `SI`) and provides patient-level evaluation metrics.


In [ ]:
import os
import random
from dataclasses import dataclass
from typing import Dict, List, Tuple
from collections import defaultdict

import numpy as np
import pandas as pd
from PIL import Image

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torchvision.transforms import functional as F
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score, roc_curve, classification_report

plt.style.use("seaborn-v0_8")
print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {models.__version__ if hasattr(models, '__version__') else 'N/A'}")


In [ ]:
# Environment detection and path configuration

def is_google_colab() -> bool:
    try:
        import google.colab  # type: ignore  # noqa: F401

        return True
    except ImportError:
        return False


def is_google_drive_mounted() -> bool:
    return os.path.exists("/content/drive/MyDrive")


def configure_paths() -> Tuple[Dict[str, str], str]:
    if is_google_colab() and is_google_drive_mounted():
        base = "/content/drive/MyDrive/Hikaru_Colab_Workspace/TUPIL_Kidney"
        print("Running on Google Colab with Google Drive mounted")
    elif is_google_colab():
        base = "/content"
        print("Running on Google Colab without Google Drive mounted")
    else:
        base = "/Users/hikaru/Desktop/TUPIL/Code/TUPIL_Kidney"
        print("Running locally")

    image_folders = {
        "B_mode": os.path.join(base, "data", "lanczos_shape_corrected_only_nc_resized_images"),
        "MBF": os.path.join(base, "data", "MBF"),
        "SI": os.path.join(base, "data", "SI"),
    }
    csv_file = os.path.join(base, "csv", "patient_eGFR_at_pocus_2025_Jul_polynomial_estimation.csv")

    return image_folders, csv_file


IMAGE_FOLDERS, CSV_FILE = configure_paths()
print("Selected image folders:")
for key, value in IMAGE_FOLDERS.items():
    print(f"  {key}: {value}")
print(f"CSV file: {CSV_FILE}")


In [ ]:
# Patient data structures and loaders

@dataclass
class Patient:
    patient_id: int
    egfr_label: int
    egfr_value: float
    image_paths: Dict[str, List[str]]

    def get_paths(self, input_type: str) -> List[str]:
        return self.image_paths.get(input_type, [])

    def min_images_across_inputs(self, input_types: List[str]) -> int:
        return min(len(self.get_paths(input_type)) for input_type in input_types)


def extract_patient_id(filename: str, input_type: str) -> int:
    if input_type == "B_mode":
        # Expected format: Patient_100_Resized_Image_1.png
        return int(filename.split("_")[1])
    # Expected format: P100_PTONR_01_Image_1_rf_MBF_resized.png
    return int(filename.split("_")[0][1:])


def load_patients(
    input_types: List[str],
    image_folders: Dict[str, str],
    csv_file: str,
) -> List[Patient]:
    egfr_data = pd.read_csv(csv_file)
    egfr_data.rename(
        columns={"Patient ID": "patient_id", "eGFR (abs/closest)": "eGFR"}, inplace=True
    )
    egfr_data["patient_id"] = egfr_data["patient_id"].astype(int)
    egfr_data.set_index("patient_id", inplace=True)

    patient_image_map: Dict[int, Dict[str, List[str]]] = defaultdict(
        lambda: {input_type: [] for input_type in input_types}
    )

    for input_type in input_types:
        folder_path = image_folders.get(input_type)
        if not folder_path or not os.path.exists(folder_path):
            print(f"Warning: folder not found for {input_type}: {folder_path}")
            continue

        for filename in sorted(os.listdir(folder_path)):
            try:
                patient_id = extract_patient_id(filename, input_type)
            except Exception:
                continue

            if patient_id in egfr_data.index:
                patient_image_map[patient_id][input_type].append(
                    os.path.join(folder_path, filename)
                )

    patients: List[Patient] = []
    for patient_id, paths_dict in patient_image_map.items():
        if all(len(paths_dict[input_type]) > 0 for input_type in input_types):
            egfr_value = float(egfr_data.loc[patient_id, "eGFR"])
            egfr_label = 1 if egfr_value >= 60 else 0
            patients.append(
                Patient(
                    patient_id=patient_id,
                    egfr_label=egfr_label,
                    egfr_value=egfr_value,
                    image_paths=paths_dict,
                )
            )

    return patients


def summarize_patients(patients: List[Patient], input_types: List[str]) -> None:
    print(f"Number of patients: {len(patients)}")
    for input_type in input_types:
        counts = [len(p.get_paths(input_type)) for p in patients]
        if not counts:
            continue
        total_images = sum(counts)
        print(
            f"{input_type}: total={total_images}, min={min(counts)}, max={max(counts)}, avg={np.mean(counts):.2f}"
        )


In [ ]:
# Flexible dataset with synchronized augmentation

class FlexiblePatientDataset(Dataset):
    def __init__(
        self,
        patients: List[Patient],
        input_types: List[str],
        image_size: int = 224,
        augment: bool = False,
    ) -> None:
        self.patients = patients
        self.input_types = input_types
        self.image_size = image_size
        self.augment = augment

        self.samples: List[Tuple[List[str], int, int]] = []
        for patient in patients:
            n_images = patient.min_images_across_inputs(input_types)
            for idx in range(n_images):
                sample_paths = [patient.get_paths(input_type)[idx] for input_type in input_types]
                self.samples.append((sample_paths, patient.egfr_label, patient.patient_id))

        self.base_mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        self.base_std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

    def __len__(self) -> int:
        return len(self.samples)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor, int]:
        image_paths, label, patient_id = self.samples[idx]
        images = [self._load_image(path) for path in image_paths]

        if self.augment:
            images = self._apply_augmentations(images)

        tensors = [F.to_tensor(img) for img in images]
        tensors = [self._normalize_tensor(t) for t in tensors]
        concatenated = torch.cat(tensors, dim=0)

        return concatenated, torch.tensor(label, dtype=torch.float32), patient_id

    def _load_image(self, path: str) -> Image.Image:
        image = Image.open(path).convert("RGB")
        return F.resize(image, [self.image_size, self.image_size])

    def _normalize_tensor(self, tensor: torch.Tensor) -> torch.Tensor:
        base_mean = self.base_mean.to(tensor.dtype)
        base_std = self.base_std.to(tensor.dtype)
        return (tensor - base_mean) / base_std

    def _apply_augmentations(self, images: List[Image.Image]) -> List[Image.Image]:
        # Match the TensorFlow pipeline: Random horizontal flip, rotation (±0.25 rad ≈ 14°), and zoom (±10%)
        do_flip = random.random() < 0.5
        angle_deg = random.uniform(-14.0, 14.0)
        zoom = random.uniform(0.9, 1.1)

        augmented = []
        for image in images:
            if do_flip:
                image = F.hflip(image)
            image = F.affine(image, angle=angle_deg, translate=(0, 0), scale=zoom, shear=0.0)
            augmented.append(image)
        return augmented


In [ ]:
# Model builder: ResNet18 with flexible input channels

def build_resnet18(num_input_channels: int, device: torch.device) -> nn.Module:
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    if num_input_channels != 3:
        old_conv = model.conv1
        model.conv1 = nn.Conv2d(
            num_input_channels,
            old_conv.out_channels,
            kernel_size=old_conv.kernel_size,
            stride=old_conv.stride,
            padding=old_conv.padding,
            bias=old_conv.bias is not None,
        )
        nn.init.kaiming_normal_(model.conv1.weight, mode="fan_out", nonlinearity="relu")
        if old_conv.bias is not None:
            nn.init.zeros_(model.conv1.bias)
    model.fc = nn.Linear(model.fc.in_features, 1)
    return model.to(device)


def count_trainable_parameters(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [ ]:
# Training utilities

def set_seeds(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def compute_pos_weight(labels: List[int]) -> float:
    positives = sum(labels)
    negatives = len(labels) - positives
    if positives == 0:
        return 1.0
    return max(1.0, negatives / max(positives, 1))


def train_one_epoch(
    model: nn.Module,
    dataloader: DataLoader,
    criterion: nn.Module,
    optimizer: torch.optim.Optimizer,
    device: torch.device,
) -> Tuple[float, float]:
    model.train()
    total_loss = 0.0
    all_labels: List[float] = []
    all_probs: List[float] = []

    for images, labels, _ in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = model(images).squeeze(1)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        probs = torch.sigmoid(logits).detach().cpu().numpy()
        all_probs.extend(probs.tolist())
        all_labels.extend(labels.detach().cpu().numpy().tolist())
        total_loss += loss.item() * images.size(0)

    epoch_loss = total_loss / len(dataloader.dataset)
    epoch_auc = roc_auc_score(all_labels, all_probs) if len(set(all_labels)) > 1 else float("nan")
    return epoch_loss, epoch_auc


def evaluate(
    model: nn.Module,
    dataloader: DataLoader,
    criterion: nn.Module,
    device: torch.device,
) -> Tuple[float, float, Dict[int, List[float]], Dict[int, int]]:
    model.eval()
    total_loss = 0.0
    all_labels: List[float] = []
    all_probs: List[float] = []
    patient_probs: Dict[int, List[float]] = defaultdict(list)
    patient_labels: Dict[int, int] = {}

    with torch.no_grad():
        for images, labels, patient_ids in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            logits = model(images).squeeze(1)
            loss = criterion(logits, labels)

            probs = torch.sigmoid(logits).cpu().numpy()
            total_loss += loss.item() * images.size(0)

            labels_np = labels.cpu().numpy()
            patient_ids_np = patient_ids.numpy()

            all_probs.extend(probs.tolist())
            all_labels.extend(labels_np.tolist())

            for pid, prob, lab in zip(patient_ids_np, probs, labels_np):
                patient_probs[int(pid)].append(float(prob))
                patient_labels[int(pid)] = int(lab)

    epoch_loss = total_loss / len(dataloader.dataset)
    epoch_auc = roc_auc_score(all_labels, all_probs) if len(set(all_labels)) > 1 else float("nan")
    return epoch_loss, epoch_auc, patient_probs, patient_labels


def plot_roc_curves(
    probs: List[float],
    labels: List[int],
    patient_probs: Dict[int, List[float]],
    patient_labels: Dict[int, int],
    title_suffix: str,
) -> Tuple[float, float]:
    if len(set(labels)) > 1:
        fpr_img, tpr_img, _ = roc_curve(labels, probs)
        auc_img = roc_auc_score(labels, probs)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr_img, tpr_img, label=f"Image ROC (AUC={auc_img:.4f})", color="tab:blue")
        plt.plot([0, 1], [0, 1], "k--", label="Random")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title(f"Image-level ROC {title_suffix}")
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        auc_img = float("nan")

    patient_mean_probs = [np.mean(patient_probs[pid]) for pid in patient_probs]
    patient_true = [patient_labels[pid] for pid in patient_probs]

    if len(set(patient_true)) > 1:
        fpr_pat, tpr_pat, _ = roc_curve(patient_true, patient_mean_probs)
        auc_pat = roc_auc_score(patient_true, patient_mean_probs)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr_pat, tpr_pat, label=f"Patient ROC (AUC={auc_pat:.4f})", color="tab:green")
        plt.plot([0, 1], [0, 1], "k--", label="Random")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title(f"Patient-level ROC {title_suffix}")
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        auc_pat = float("nan")

    return auc_img, auc_pat


In [ ]:
# Experiment configuration

INPUT_TYPES = ["B_mode"]  # e.g., ["B_mode", "MBF"], ["B_mode", "MBF", "SI"]
IMAGE_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 100
EARLY_STOPPING_PATIENCE = 20
LEARNING_RATE = 1e-4
STEP_LR_EVERY = 15
STEP_LR_GAMMA = 0.5
N_RUNS = 5
BASE_SEED = 42

set_seeds(BASE_SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"Selected input types: {INPUT_TYPES}")


In [ ]:
# Load and summarize patient data

all_patients = load_patients(INPUT_TYPES, IMAGE_FOLDERS, CSV_FILE)
if not all_patients:
    raise RuntimeError("No patients found with the specified input types.")

summarize_patients(all_patients, INPUT_TYPES)


In [ ]:
# Hold-out runs with training, validation, and testing

results = []

for run_idx in range(N_RUNS):
    seed = BASE_SEED + run_idx
    set_seeds(seed)

    print("=" * 60)
    print(f"Run {run_idx + 1}/{N_RUNS} (seed={seed})")
    print("=" * 60)

    train_val_patients, test_patients = train_test_split(all_patients, test_size=0.1, random_state=seed)
    train_patients, val_patients = train_test_split(train_val_patients, test_size=0.2, random_state=seed)

    print(f"Train patients: {len(train_patients)}")
    print(f"Validation patients: {len(val_patients)}")
    print(f"Test patients: {len(test_patients)}")

    train_dataset = FlexiblePatientDataset(train_patients, INPUT_TYPES, image_size=IMAGE_SIZE, augment=True)
    val_dataset = FlexiblePatientDataset(val_patients, INPUT_TYPES, image_size=IMAGE_SIZE, augment=False)
    test_dataset = FlexiblePatientDataset(test_patients, INPUT_TYPES, image_size=IMAGE_SIZE, augment=False)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    model = build_resnet18(num_input_channels=3 * len(INPUT_TYPES), device=device)
    print(f"Trainable parameters: {count_trainable_parameters(model):,}")

    train_labels = [label for _, label, _ in train_dataset.samples]
    pos_weight_value = compute_pos_weight(train_labels)
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weight_value, device=device))
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_LR_EVERY, gamma=STEP_LR_GAMMA)

    best_val_auc = -1.0
    best_state = None
    epochs_without_improvement = 0

    for epoch in range(1, EPOCHS + 1):
        train_loss, train_auc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_auc, val_patient_probs, val_patient_labels = evaluate(model, val_loader, criterion, device)
        scheduler.step()

        print(
            f"Epoch {epoch:3d} | Train Loss: {train_loss:.4f} AUC: {train_auc:.4f} | "
            f"Val Loss: {val_loss:.4f} AUC: {val_auc:.4f}"
        )

        if val_auc > best_val_auc:
            best_val_auc = val_auc
            best_state = {
                "model": model.state_dict(),
                "optimizer": optimizer.state_dict(),
            }
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= EARLY_STOPPING_PATIENCE:
            print("Early stopping triggered.")
            break

    if best_state is None:
        raise RuntimeError("Training did not yield a valid model state.")

    model.load_state_dict(best_state["model"])

    val_loss, val_auc, val_patient_probs, val_patient_labels = evaluate(model, val_loader, criterion, device)
    val_probs_flat = [prob for probs in val_patient_probs.values() for prob in probs]
    val_labels_flat = [val_patient_labels[pid] for pid, probs in val_patient_probs.items() for _ in probs]
    val_img_auc, val_patient_auc = plot_roc_curves(
        val_probs_flat,
        val_labels_flat,
        val_patient_probs,
        val_patient_labels,
        title_suffix=f"(Validation Run {run_idx + 1})",
    )

    test_loss, test_auc, test_patient_probs, test_patient_labels = evaluate(model, test_loader, criterion, device)
    test_probs_flat = [prob for probs in test_patient_probs.values() for prob in probs]
    test_labels_flat = [test_patient_labels[pid] for pid, probs in test_patient_probs.items() for _ in probs]
    test_img_auc, test_patient_auc = plot_roc_curves(
        test_probs_flat,
        test_labels_flat,
        test_patient_probs,
        test_patient_labels,
        title_suffix=f"(Test Run {run_idx + 1})",
    )

    preds_binary = [1 if prob >= 0.5 else 0 for prob in test_probs_flat]
    print("Test classification report (image-level):")
    print(classification_report(test_labels_flat, preds_binary, digits=4))

    results.append(
        {
            "val_auc": val_auc,
            "test_auc": test_auc,
            "val_img_auc": val_img_auc,
            "val_patient_auc": val_patient_auc,
            "test_img_auc": test_img_auc,
            "test_patient_auc": test_patient_auc,
        }
    )


In [ ]:
# Aggregate results across runs

if results:
    print("\n" + "=" * 60)
    print("FINAL SUMMARY OVER MULTIPLE HOLD-OUT RUNS")
    print("=" * 60)
    val_aucs = [entry["val_auc"] for entry in results]
    test_aucs = [entry["test_auc"] for entry in results]
    print(f"Validation AUCs: {[f'{auc:.4f}' for auc in val_aucs]}")
    print(f"Test AUCs:       {[f'{auc:.4f}' for auc in test_aucs]}")
    print(f"\nAverage Validation AUC: {np.mean(val_aucs):.4f} ± {np.std(val_aucs):.4f}")
    print(f"Average Test AUC:       {np.mean(test_aucs):.4f} ± {np.std(test_aucs):.4f}")
else:
    print("No runs executed yet.")


In [ ]:
# Cross-validation testing (patient-level K-Fold)

N_FOLDS = 5
cv_results = []

kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=BASE_SEED)

for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(all_patients)):
    print("=" * 60)
    print(f"Cross-validation fold {fold_idx + 1}/{N_FOLDS}")
    print("=" * 60)

    test_patients = [all_patients[i] for i in test_idx]
    train_val_patients = [all_patients[i] for i in train_val_idx]

    rng = np.random.default_rng(BASE_SEED + fold_idx)
    rng.shuffle(train_val_patients)
    val_split = max(1, int(len(train_val_patients) * 0.2))
    val_patients = train_val_patients[:val_split]
    train_patients = train_val_patients[val_split:]

    print(f"Training patients: {len(train_patients)}")
    print(f"Validation patients: {len(val_patients)}")
    print(f"Test patients: {len(test_patients)}")

    train_dataset = FlexiblePatientDataset(train_patients, INPUT_TYPES, image_size=IMAGE_SIZE, augment=True)
    val_dataset = FlexiblePatientDataset(val_patients, INPUT_TYPES, image_size=IMAGE_SIZE, augment=False)
    test_dataset = FlexiblePatientDataset(test_patients, INPUT_TYPES, image_size=IMAGE_SIZE, augment=False)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    model = build_resnet18(num_input_channels=3 * len(INPUT_TYPES), device=device)
    print(f"Trainable parameters: {count_trainable_parameters(model):,}")

    train_labels = [label for _, label, _ in train_dataset.samples]
    pos_weight_value = compute_pos_weight(train_labels)
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weight_value, device=device))
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_LR_EVERY, gamma=STEP_LR_GAMMA)

    best_val_auc = -1.0
    best_state = None
    epochs_without_improvement = 0

    for epoch in range(1, EPOCHS + 1):
        train_loss, train_auc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_auc, val_patient_probs, val_patient_labels = evaluate(model, val_loader, criterion, device)
        scheduler.step()

        print(
            f"Epoch {epoch:3d} | Train Loss: {train_loss:.4f} AUC: {train_auc:.4f} | "
            f"Val Loss: {val_loss:.4f} AUC: {val_auc:.4f}"
        )

        if val_auc > best_val_auc:
            best_val_auc = val_auc
            best_state = {
                "model": model.state_dict(),
                "optimizer": optimizer.state_dict(),
            }
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= EARLY_STOPPING_PATIENCE:
            print("Early stopping triggered.")
            break

    if best_state is None:
        raise RuntimeError("Training did not yield a valid model state.")

    model.load_state_dict(best_state["model"])

    val_loss, val_auc, val_patient_probs, val_patient_labels = evaluate(model, val_loader, criterion, device)
    val_probs_flat = [prob for probs in val_patient_probs.values() for prob in probs]
    val_labels_flat = [val_patient_labels[pid] for pid, probs in val_patient_probs.items() for _ in probs]
    val_img_auc, val_patient_auc = plot_roc_curves(
        val_probs_flat,
        val_labels_flat,
        val_patient_probs,
        val_patient_labels,
        title_suffix=f"(Validation Fold {fold_idx + 1})",
    )

    test_loss, test_auc, test_patient_probs, test_patient_labels = evaluate(model, test_loader, criterion, device)
    test_probs_flat = [prob for probs in test_patient_probs.values() for prob in probs]
    test_labels_flat = [test_patient_labels[pid] for pid, probs in test_patient_probs.items() for _ in probs]
    test_img_auc, test_patient_auc = plot_roc_curves(
        test_probs_flat,
        test_labels_flat,
        test_patient_probs,
        test_patient_labels,
        title_suffix=f"(Test Fold {fold_idx + 1})",
    )

    preds_binary = [1 if prob >= 0.5 else 0 for prob in test_probs_flat]
    print("Test classification report (image-level):")
    print(classification_report(test_labels_flat, preds_binary, digits=4))

    cv_results.append(
        {
            "val_auc": val_auc,
            "test_auc": test_auc,
            "val_img_auc": val_img_auc,
            "val_patient_auc": val_patient_auc,
            "test_img_auc": test_img_auc,
            "test_patient_auc": test_patient_auc,
        }
    )


In [ ]:
# Cross-validation summary

if cv_results:
    print("\n" + "=" * 60)
    print("FINAL CROSS-VALIDATION SUMMARY")
    print("=" * 60)
    cv_val_aucs = [entry["val_auc"] for entry in cv_results]
    cv_test_aucs = [entry["test_auc"] for entry in cv_results]
    print(f"Validation AUCs: {[f'{auc:.4f}' for auc in cv_val_aucs]}")
    print(f"Test AUCs:       {[f'{auc:.4f}' for auc in cv_test_aucs]}")
    print(f"\nAverage Validation AUC: {np.mean(cv_val_aucs):.4f} ± {np.std(cv_val_aucs):.4f}")
    print(f"Average Test AUC:       {np.mean(cv_test_aucs):.4f} ± {np.std(cv_test_aucs):.4f}")
else:
    print("Cross-validation has not been run yet.")
